## Air Temperature Calibration Model Development 

**Task Description**

**Goal:** \
    The goal of a calibration model is to improve the accuracy and reliability of one of our sensor’s outputs. \
**Assignment:** \
    Primary objective is to output a value for air temperature that is more accurate and reliable than the raw value from our main air temperature sensors
    
**Assessment:** \
It can take many days or weeks to develop a reliable and performant calibration model prototype. This assignment will not be assessed by the performance of the final model. It will be assessed on:
-	Demonstrating a thorough understanding of the problem space
-	Proficiency in data handling and manipulation
-	Demonstrating a structured model development process
-	Effectively evaluating model(s)/solution(s) 
-	Ability to communicate process and findings
-	A performant model is a plus!


In [ ]:
import numpy as np
import pandas as pd

###need to check
from datetime import datetime
from pytz import timezone
import pytz

# Import packages for model building
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, PowerTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso

#Visualization
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
import seaborn as sns

In [ ]:
dic=pd.read_csv('data_dictionary.csv')

In [ ]:
##reference file 
# dic=dic[['File','Name','Description']]
# dic

In [ ]:
#sensor dataframe
df=pd.read_csv('mark_data.csv')

### Data OVerView

In [ ]:
df.head(2)

In [ ]:
# df.info()

In [ ]:
# check duplicate records: No duplicates
df.duplicated().any()

In [ ]:
#check missing value: lwdw are mostly missing could be removed
# df.isnull().sum()

### drop lwdw 

In [ ]:
#drop missing column
df.drop(['lwdw'],axis=1, inplace=True)

### Independent variable

In [ ]:
#change time to timestamp
df.time = pd.to_datetime(df.time, errors='coerce')
# df.info()

In [ ]:
df.dqs.value_counts()
#dqs can be convert to category data type

In [ ]:
df.dqs=pd.Categorical(df.dqs)

In [ ]:
df.device.value_counts()
#there are 5 unique devices could be used to group data 
df.device=pd.Categorical(df.device)

In [ ]:
#group data by categories for later modeling
float_v = list(df.select_dtypes(include=['float64']).columns)
cat_v = list(df.select_dtypes(include=['category']).columns)

In [ ]:
# float_v, cat_v

### change time to index

In [ ]:
df=df.set_index('time')

In [ ]:
# df[df['dqs']==4]['b1dw'].plot()
# cannot find dqs description and move ahead with caution now

In [ ]:
df.dqs.value_counts()

In [ ]:
device_name=df.device.value_counts().reset_index()

cats=list(device_name.iloc[:,0]) 
cats

In [ ]:
device_name=df.device.value_counts().reset_index()

cats=list(device_name.iloc[:,0]) 
cats

### Batch visualization 

In [ ]:
#let's plot data in batch to QC
def quick_look(df, cat_name, cats, props):
    '''
    plot all property:props plots by cats:category
    '''
    for cat in cats:
        fig, ax = plt.subplots()
        plt.plot(df[df[cat_name]==cat][props])
        ax.set_xlabel(cats, fontsize=15)
        ax.set_ylabel(props, fontsize=15)
quick_look(df, 'device', cats, 'temp')

***Note**: there are some outlier 60C C004894 ?

### some outlier plots

In [ ]:
def plot_temps(df, device, prop):
    '''
    input dataframe, category(such as devices)
    property(such as temprature)
    '''
    
    plt.plot(df[df.device==device][prop])
    ax.set_xlabel( device, fontsize=15)
    ax.set_ylabel(prop, fontsize=15)



In [ ]:
# capture some outliers plots
fig, ax = plt.subplots()
plot_temps(df,'C004894','therm_temp')


In [ ]:
plot_temps(df,'C004894','temp' )

In [ ]:
plot_temps(df,'C005348','therm_temp' )

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)

    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range

    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
#     print(q1, q3, fence_low,fence_high)
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
#remove outliers simplified approach. could you 3 sigma approach. After tested no difference in this dataset
#outlier handling should be build in pipeline to streamline for deployment
float_v_outliers=['temp', 'temp_sd', 'therm_temp', 'therm_temp_sd', 'p', 't_u14']
for feature in float_v_outliers:
    df = remove_outlier(df, feature)
    




In [ ]:
plt.plot(df[df.device=='C004894']['p'])

### Uni Variate Analysis


### plot histograms for quick look

In [ ]:
ncols = 4
# nrows=2
nrows = int(np.ceil(len(df.columns) / (1.0*ncols)))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 12))

# Lazy counter so we can remove unwated axes
counter = 0
for i in range(nrows):
    for j in range(ncols):

        ax = axes[i][j]

        # Plot when we have data
        if counter < len(df.columns):

            ax.hist(df[df.columns[counter]], bins=20, color='red', alpha=0.5, label='{}'.format(df.columns[counter]))
#             ax.set_xlabel('x')
#             ax.set_ylabel('PDF')
#             ax.set_ylim([0, 5])
            leg = ax.legend(loc='upper right')
            leg.draw_frame(False)

        # Remove axis when we no longer have data
        else:
            ax.set_axis_off()

        counter += 1

plt.show()

### boxplot  for quick look

In [ ]:
sns.set(rc={'figure.figsize':(16,8.27)})
fig.tight_layout()
# float_v.remove('lwdw')
# float_v.remove('p')

sns.boxplot(data=df[float_v])

***Note:*** lwuw, swdw, swuw still has significant outliers

### Independent variables

In [ ]:
df_ref=pd.read_csv('reference_data.csv')

In [ ]:
#create a dictionary to map site with device

devSite={'AZ_LTER': 'C004991',
 'NREL': 'C004988',
 'ESALQ': 'C004894',
 'US-Los': 'C006826',
 'US-Seg': 'C005348'}

#### create new column device in reference df

In [ ]:
df_ref['device']=df_ref.site_id.apply(lambda x: devSite[x] if x in devSite else x )

#### convert types: time, object

In [ ]:
df_ref.ref_time = pd.to_datetime(df_ref.ref_time, errors='coerce')
df_ref.device=pd.Categorical(df_ref.device)
df_ref.site_id=pd.Categorical(df_ref.site_id)

In [ ]:
df_ref.rename(columns={'ref_time':'time'}, inplace=True)

In [ ]:
df_ref.set_index('time', inplace=True)

In [ ]:
df_ref_copy=df_ref.copy()

In [ ]:
# df_ref_copy.groupby('device').resample('D')['ref_tair'].mean()

In [ ]:
# # df_ref_copy.ref_tair.plot()
# fig = plt.figure(figsize=(12,4)) 

# sns.lineplot(data=df_ref_copy.ref_tair, color='blue', linewidth=1)

#### join mark data and df_ref using device and time to create training data

In [ ]:
df_copy=df.copy()
#this step is time consuming activate when necessary
# df_new=pd.merge(df_ref, df_copy,  how='right', on=['time','device'])
#https://towardsdatascience.com/how-to-merge-not-matching-time-series-with-pandas-7993fcbce063
df_ref = df_ref.sort_values(['time'])
df_copy = df_copy.sort_values(['time'])
df_new=pd.merge_asof(df_ref, df_copy,  on='time', by='device',tolerance=pd.Timedelta('10min'))

In [ ]:
# df_new.info()
df_ref.shape, df_new.shape, df_copy.shape

***Note:*** looks like we are able to get most data matched and assigned. lets remove nulls

In [ ]:
pd.set_option('display.max_columns', None)
# more options can be specified also
df_new1=df_new[~df_new['ref_tair'].isnull()& ~df_new['temp'].isnull()]

In [ ]:
df_new1.set_index('time', inplace=True)

In [ ]:

df_new1.describe()

In [ ]:
df_new1.shape

#### We miss significant portion of data during merging even though we tried timedelta approach.

#### We may want to revisit before deadline. Now let's move on!

In [ ]:
#df_new1.describe()
#df_new1.info()
# df_new1.isnull().sum()

***Note*** Let's create a heatmap to check dependent and independent variables relationship 

In [ ]:
df_new1.columns
#group data by categories for later modeling
float_v = list(df_new1.select_dtypes(include=['float64']).columns)
cat_v = list(df_new1.select_dtypes(include=['category']).columns)
cat_v

### Bivariate Analysis

In [ ]:
# printmd('**independent and dependent variables, which features have high correlation with dependent var. and how about colinearlity?**')
sns.set(style="white")
features=float_v
corr = df_new1[features].corr()

#Generate a mask for the upper triangle:
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
#Set up the matplotlib figure and a diverging colormap:
f, ax = plt.subplots(figsize=(18, 15))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
#Draw the heatmap with the mask and correct aspect ratio:
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.9, center=0, annot=False, 
square=True, linewidths=.5, cbar_kws={"shrink": .5})

***Note:*** 
1. very strong colinearalities amond bxdw features
2. strong colinearlities among temprature measurements
3. colinearlity should be handled at modeling stage
4. reference temprature has strong correlation with temp, therm_temp, lwuw,swuw, t_u14, t_u21

# 1. Dummy Transformer for Categorical Data


In [ ]:
#since site_id and device are one to one match, we can drop device
df_new1.drop(['device'],axis=1, inplace=True)

In [ ]:
df_new1.shape

In [ ]:

# cat_v.remove('device')
cat_features=['site_id', 'dqs']
df_new1 = pd.get_dummies(df_new1, columns=cat_features, drop_first=True)

In [ ]:
df_new1.head()

# 1. Random Forest

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#### random split to train test data

In [ ]:

# features
features=df_new1.columns.tolist()
target='ref_tair'
features.remove(target)


In [ ]:
features=df_new1.columns.tolist()
target='ref_tair'
features.remove(target)
X=df_new1[features]
y=df_new1[target]

# split into train/test sets with same class ratio
X_train, X_test, y_train, y_test =    train_test_split(X, y, test_size=0.3,
                     random_state=42)


In [ ]:
X_train.shape

In [ ]:
steps=[('rescale', MinMaxScaler()),
       ('pca', PCA(n_components=10)),
       ('lr', Lasso(alpha=1, max_iter=100000))]
#        ('rfr', RandomForestRegressor(random_state=0))]
pipe=Pipeline(steps)
pipe = pipe.fit(X_train, y_train)

In [ ]:
y_train_pred=pipe.predict(X_train)
y_test_pred=pipe.predict(X_test)

In [ ]:
print('train MAE: {0:.2e}'.format(mean_absolute_error(y_train, y_train_pred)))
print('train MSE: {0:.2e}'.format(mean_squared_error(y_train, y_train_pred)))
print('train R2: {0:.3f}'.format(r2_score(y_train, y_train_pred)))

print('test MAE: {0:.2e}'.format(mean_absolute_error(y_test, y_test_pred)))
print('test MSE: {0:.2e}'.format(mean_squared_error(y_test, y_test_pred)))
print('test R2: {0:.3f}'.format(r2_score(y_test, y_test_pred)))

In [ ]:
feat_importances = pd.Series(pipe.steps[1][1].feature_importances_, index=X_train.columns)
feat_importances.sort_values(ascending=False).plot(kind='bar')

In [ ]:
# avgdf = (df_new1.reset_index()
#           .groupby(['time','device'], as_index=False)
#           .mean()
# #           # rename isn't strictly necessary here, it's just for readability
# #           .rename(columns={'index':'ct'})
#        )

In [ ]:
# avgdf

In [ ]:
# fig, ax = plt.subplots()

# # key gives the group name (i.e. category), data gives the actual values
# for key, data in df_new1.groupby('device'):
#     data.plot(x='time', y='ref_tair', ax=ax, label=key)

In [ ]:
# plt.plot(df_new1.groupby(['device'])['ref_tair'])
# # plt.plot(df_new1['temp'])
# # plt.plot(df_new1['temp'])